In [1]:
import os
import openai
import requests
from pprint import pprint


openai_api_key = ".............."
bing_search_api_key = '................'
bing_search_endpoint = '...........' + \
    "v7.0/search"

def search(query):
    # Construct a request
    mkt = 'en-US'
    params = {'q': query, 'mkt': mkt}
    headers = {'Ocp-Apim-Subscription-Key': bing_search_api_key}

    # Call the API
    try:
        response = requests.get(bing_search_endpoint,
                                headers=headers, params=params)
        response.raise_for_status()
        json = response.json()
        return json["webPages"]["value"]

        # print("\nJSON Response:\n")
        # pprint(response.json())
    except Exception as ex:
        raise ex


# Prompt the user for a question
question = input("Ask me? ")

# Send a query to the Bing search engine and retrieve the results
results = search(question)

results_prompts = [
    f"Source:\nTitle: {result['name']}\nURL: {result['url']}\nContent: {result['snippet']}" for result in results
]

prompt = "Use these sources to answer the question:\n\n" + \
    "\n\n".join(results_prompts) + "\n\nQuestion: " + question + "\n\nAnswer:"

print(prompt)

# Check if there are any results
if results:
    # Use OpenAI's GPT-3 API to answer the question
    openai.api_key ="....................."
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages = [{"role":"user","content":prompt}],
        temperature=0.7,
        max_tokens=4000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    # Print the answer from OpenAI
    response = response["choices"][0]["message"]['content']
    print(f"Answer: {response}")
else:
    # Print an error message if there are no results
    print("Error: No results found for the given query.")